# Random Forest demo
This is a demo for my random forest algorithm.  I'm using a very small data set, so the predictions will not be very good, but it will at least demonstrate how to use the algorithm.

In [1]:
import pandas as pd
import numpy as np
import sys
import random

## Demo data
I've included a small set of demo data, not really enough to produce a good model, but enough to demonstrate the potential of the algorithm.  This comes from a physical source, so that the labels should be predictable, but I've anonymized the data to obscure the data's meaning.  There are 500 entries with 40 features each, and one label.  About 80% of the labels are zeros, and 20% are ones.

In [2]:
data = pd.read_pickle("demo_data.pkl")

In [3]:
#Data includes 500 entries, 40 feature columns, and 1 label column
print(data.shape)

(500, 41)


In [4]:
data.head()

,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,...,feature 31,feature 32,feature 33,feature 34,feature 35,feature 36,feature 37,feature 38,feature 39,label
0,1860.0,42.0,3,15,139,6146.0,8.60839,0.000,-4.927546,-4.117362,...,15,0,0,0,0,3,51,6,14.0,1.0
1,4300.0,33.0,12,14,76,7140.0,11.26030,4.496,-1.240000,-9.209714,...,4,0,0,0,0,2,6,4,43.0,0.0
2,4273.0,320.0,12,13,84,8650.0,8.29802,1.524,-0.812174,-6.677698,...,3,0,0,0,0,1,5,5,19.0,0.0
3,3675.0,48.0,5,3,187,6110.0,5.42586,0.000,-8.939261,-4.523000,...,17,9,0,0,14,0,71,4,47.0,0.0
4,887.0,22.0,10,15,119,8908.0,9.78860,0.000,-5.546325,-4.650889,...,15,0,0,0,0,3,33,4,76.0,0.0


In [5]:
#I will separate the data into training data, with 400 rows, and test data, with 100 rows.
subset = np.random.choice(range(500),100,replace=False)
test_data = data.iloc[subset]
train_data = data.drop(index=subset,axis=0)

## Single trees
The user is not intended to grow single trees (since they just get overtrained very quickly), but it may be instructive to show how a single tree is grown.

In [6]:
#load decisionTrees module
if 'decision_trees' in sys.modules:
    del sys.modules['decision_trees']
from decision_trees import *

In [7]:
#initialize the tree, telling it which columns are features, and which one is the label
single_tree = DecisionTree(train_data,range(40),40, 40)

In [8]:
#Watch the tree grow 5 leaves.
single_tree.verbose = True
single_tree.grow_tree(train_data,5,0)

Leaf 0 produces children using column 26.
Weighted error in this branch: 96.0 before; 3483.0 after
Leaf 1 produces children using column 1.
Weighted error in this branch: 3600.0 before; 3362.0 after
Leaf 1 produces children using column 31.
Weighted error in this branch: 3440.0 before; 2610.0 after
Leaf 1 produces children using column 33 (chosen at random).
Weighted error in this branch: 3000.0 before; 3000.0 after
Leaf 1 produces children using column 39 (chosen at random).
Weighted error in this branch: 2440.0 before; 2440.0 after


An explanation:

At each iteration, the tree determines which leave makes the most incorrect predictions, and tells that leaf to produce new children.  The leaf selects the column using a greedy algorithm to maximize correct predictions, and produces children using that column.  If predictions cannot be improved, then the leaf chooses a column at random.  (As you can see, not much improvement is made in the first five iterations.)

In [9]:
#You can also print out the tree structure.
#Each leaf is labeled a "0" or "1" depending on how it classifies data points that reach that leaf of the tree
#as you can see, the tree does not try to be balanced.
print(single_tree.root.summarize())

#And you can evaluate how well the decision tree makes predictions (not very well)
print('Accuracy on training data: %.2f' % single_tree.evaluate(train_data))
print('Accuracy on test data: %.2f' % single_tree.evaluate(test_data))

[1,[[[[0,0],0],1],1]]
Accuracy on training data: 0.78
Accuracy on test data: 0.76


In [10]:
#Show what happens after 50 more iterations
single_tree.verbose = False
single_tree.grow_tree(train_data,50,0)
print(single_tree.root.summarize())
print('Accuracy on training data: %.2f' % single_tree.evaluate(train_data))
print('Accuracy on test data: %.2f' % single_tree.evaluate(test_data))

[1,[[[[[[[[[[1,0],0],1],[1,[[[0,0],1],[1,[[0,1],1]]]]],[1,[[[[[[0,[0,1]],0],[1,0]],[0,1]],0],1]]],[[0,1],1]],[[[[1,[1,[0,0]]],1],[[1,0],[0,0]]],[[[[1,[[0,0],[[[[[1,0],0],0],[1,0]],1]]],[1,0]],0],1]]],[1,[[0,1],1]]],1],1]]
Accuracy on training data: 0.87
Accuracy on test data: 0.65


# Random forest testing
Now let's show how a whole forest is grown.  Each tree is given a random subset of rows (selected with replacement) and a random subset of feature columns.

In [11]:
#See the readme for details on these arguments
#This forest will grow trees, each given 5 features and 50 iterations,
#and keep on growing trees until 1 minute has passed.
forest = RandomForest(range(40),40,50,0,5,1)

In [12]:
#train the forest, and then show how well it performs on the test data
forest.train_valid(train_data,test_data)

Forest planted in 1.00 minutes. 110 trees grown with 50 iterations and 5 features.
Accuracy: 0.800


In [13]:
#We can also print out the confusion matrix
print(forest.confusion(test_data))

[[0.06 0.15]
 [0.05 0.74]]


In [14]:
#or we can just print out the predicted labels
pred_labels = forest.predict_labels(test_data)
for i in range(10):
    print("Row %i: Actual label: %i, Predicted label: %i" % (i, test_data['label'].iloc[i], pred_labels[i]))

Row 0: Actual label: 0, Predicted label: 0
Row 1: Actual label: 0, Predicted label: 0
Row 2: Actual label: 0, Predicted label: 0
Row 3: Actual label: 0, Predicted label: 0
Row 4: Actual label: 0, Predicted label: 0
Row 5: Actual label: 0, Predicted label: 0
Row 6: Actual label: 1, Predicted label: 1
Row 7: Actual label: 0, Predicted label: 0
Row 8: Actual label: 0, Predicted label: 0
Row 9: Actual label: 0, Predicted label: 0


In [15]:
#You can also retrieve information about any particular tree
print(forest.forest[0].evaluate(test_data))

0.6699999999999999


Given the imbalance in the data (far more zeros than ones), it might be appropriate to apply some weighting, so that the algorithm considers classification of 1s to be more important.

In [16]:
weight = 1 - (data['label'].sum() / data.shape[0])
print(weight)

0.766


In [17]:
forest = RandomForest(range(40), 40, 50, 0, 5, 1, weight=weight)
forest.train_valid(train_data,test_data)
print(forest.confusion(test_data))

Forest planted in 1.00 minutes. 105 trees grown with 50 iterations and 5 features.
Accuracy: 0.820
[[0.1  0.11]
 [0.07 0.72]]


# Hyperparameter tuning
While I did not write a framework for tuning hyperparameters, most of the tools are already there!  In particular, the forest will train for a fixed amount of time, so that you can figure out what hyperparameters are most effective given your available computing resources.

Here I use a simple search to determine whether it's better to grow more trees, or give each tree access to a larger subspace.

In [18]:
#note: this takes about 4 minutes to run
forest_grid = []
sim_index = 0
sim_results = pd.DataFrame(np.zeros((4,9)),columns=[
        'Iterations','Features','Trees','Runtime','accuracy','cmatrix_00','cmatrix_01','cmatrix_10','cmatrix_11'])
for i in range(4):
    forest = RandomForest(range(40),40,50,0,5+5*i,1,weight=weight)
    forest_grid.append(forest)
    forest.train(train_data)
    sim_results.iloc[i,0] = forest.num_iter
    sim_results.iloc[i,1] = forest.num_features
    sim_results.iloc[i,2] = forest.num_trees
    sim_results.iloc[i,3] = forest.runtime
    c_matrix = forest.confusion(test_data)
    sim_results.iloc[i,4] = c_matrix.trace()
    sim_results.iloc[i,5] = c_matrix[0,0]
    sim_results.iloc[i,6] = c_matrix[0,1]
    sim_results.iloc[i,7] = c_matrix[1,0]
    sim_results.iloc[i,8] = c_matrix[1,1]

Forest planted in 1.01 minutes. 110 trees grown with 50 iterations and 5 features.
Forest planted in 1.00 minutes. 63 trees grown with 50 iterations and 10 features.
Forest planted in 1.01 minutes. 44 trees grown with 50 iterations and 15 features.
Forest planted in 1.00 minutes. 32 trees grown with 50 iterations and 20 features.


In [19]:
sim_results

,Iterations,Features,Trees,Runtime,accuracy,cmatrix_00,cmatrix_01,cmatrix_10,cmatrix_11
0,50.0,5.0,110.0,1.009608,0.81,0.11,0.10,0.09,0.70
1,50.0,10.0,63.0,1.002941,0.78,0.12,0.09,0.13,0.66
2,50.0,15.0,44.0,1.010208,0.76,0.09,0.12,0.12,0.67
3,50.0,20.0,32.0,1.004174,0.79,0.11,0.10,0.11,0.68


It seems to work best with about 5 features.  (Of course, this result should not be taken too seriously, given the very small test data set and short runtime.)

## Comparison to scikit-learn
Here's a brief comparison to the performance of a scikit-learn model.  The parameters aren't quite the same, but I'd like to run it in a way that is as close as possible to the forest above (100 trees, 50 leaves, 5 features each, 1 minute runtime).

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import time

C:\Users\Tristan\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [21]:
unlabeled_data = train_data.iloc[:,range(40)]
labels = train_data.iloc[:,40]
unlabeled_test_data = test_data.iloc[:,range(40)]
test_labels = test_data.iloc[:,40]

In [28]:
sklearn_forest = RandomForestClassifier(n_estimators=100, max_features=5, max_leaf_nodes=50,
                                        bootstrap=True, n_jobs=1, class_weight='balanced')

In [29]:
start_time = time.time()
sklearn_forest.fit(unlabeled_data,labels)
print("Runtime: %.2f minutes" % ((time.time() - start_time)/60))
print("Accuracy: %.2f" % sklearn_forest.score(unlabeled_test_data,test_labels))

Runtime: 0.00 minutes
Accuracy: 0.84


In [30]:
#print the confusion matrix
def print_confusion(sklearn_forest, unlabeled_test_data, test_labels):
    test_predictions = sklearn_forest.predict(unlabeled_test_data)
    confusion = np.ndarray((2,2))
    confusion[:,:] = confusion_matrix(test_labels,test_predictions)
    csum = sum(sum(confusion))
    confusion /= csum
    print(confusion)
print_confusion(sklearn_forest, unlabeled_test_data, test_labels)

[[0.72 0.07]
 [0.09 0.12]]


Basically, the accuracy is similar or better, but scikit-learn is a lot more efficient in terms of computing time.  Which probably also means it can be more accurate given the same amount of computing time (although for this small data set the accuracy is about maxed out).

In [31]:
sklearn_forest = RandomForestClassifier(n_estimators=1000, max_features=5, max_leaf_nodes=500,
                                        bootstrap=True, n_jobs=1, class_weight='balanced')

In [32]:
start_time = time.time()
sklearn_forest.fit(unlabeled_data,labels)
print("Runtime: %.2f minutes" % ((time.time() - start_time)/60))
print("Accuracy: %.2f" % sklearn_forest.score(unlabeled_test_data,test_labels))

Runtime: 0.02 minutes
Accuracy: 0.85


In [33]:
print_confusion(sklearn_forest, unlabeled_test_data, test_labels)

[[0.75 0.04]
 [0.11 0.1 ]]
